# Download packages and libraries

In [ ]:
!pip install --upgrade pythainlp
!pip install emoji

     |████████████████████████████████| 8.9MB 2.6MB/s 
     |████████████████████████████████| 747kB 39.6MB/s 
     |████████████████████████████████| 51kB 2.2MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=fc371a2b4f60ee683e33d65c81c7a7c17058b24fa4a94de1e0848fee3500d9f6
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import emoji
from pythainlp.corpus.common import thai_stopwords
from pythainlp import sent_tokenize, word_tokenize
from pythainlp.ulmfit import *
from pythainlp.util import normalize
import re
import itertools
import dill as pickle

Corpus: wiki_lm_lstm
- Downloading: wiki_lm_lstm 0.32


100%|██████████| 1050919089/1050919089 [00:23<00:00, 44270453.70it/s]


Corpus: wiki_itos_lstm
- Downloading: wiki_itos_lstm 0.32


100%|██████████| 1530484/1530484 [00:00<00:00, 3906822.19it/s]


# Masterchef Data

In [ ]:
import pandas as pd
import re

In [ ]:
master_data = pd.read_csv('/content/drive/My Drive/ICT senior project/dataset/master-chef.csv')
master_data=master_data.drop(labels='Unnamed: 4', axis=1)
print(master_data.shape)
master_data.head()

(5000, 4)


,text_id,sentiment,text,date
0,1,1.0,อดเห็นชานนท์ชิมอาหารแล้ว😂 #masterchefallstarst...,22/3/2020 18:20
1,2,1.0,เอาไรกับคนในพันติ๊บ คำที่พิมเหมือนนางร้ายนางอิ...,22/3/2020 18:18
2,3,2.0,ต้องคิดถึงซีน’ชิม’ของพี่ชานนท์แน่ๆเลย มีทุกอี...,22/3/2020 18:18
3,4,1.0,ร้องไห้กับพี่ชานู้ปปี้ #masterchefallstarsthai...,22/3/2020 18:16
4,5,2.0,ชั้นรักพี่ปอนด์และรักพลังบวกนี้ แล้วทำให้ชั้นน...,22/3/2020 18:16


Positive Data

In [ ]:
master_positive = master_data[master_data['sentiment'] == 2.0]
print(master_positive.shape)
master_positive.head()

(1028, 4)


,text_id,sentiment,text,date
2,3,2.0,ต้องคิดถึงซีน’ชิม’ของพี่ชานนท์แน่ๆเลย มีทุกอี...,22/3/2020 18:18
4,5,2.0,ชั้นรักพี่ปอนด์และรักพลังบวกนี้ แล้วทำให้ชั้นน...,22/3/2020 18:16
5,6,2.0,เชียร์มาตั้งแต่ต้นจนถึงวันนี้ เห็นความตั้งใจ อ...,22/3/2020 18:16
6,7,2.0,@Pondmcss3 #MasterChefAllStarsThailandปอนด์พัฒ...,22/3/2020 18:15
8,9,2.0,เหมือนลุงข้างบ้านขิงกันอะ555555555555555555555...,22/3/2020 18:14


Negative Data

In [ ]:
master_negative = master_data[master_data['sentiment'] == 1.0]
print(master_negative.shape)
master_negative.head()

(1744, 4)


,text_id,sentiment,text,date
0,1,1.0,อดเห็นชานนท์ชิมอาหารแล้ว😂 #masterchefallstarst...,22/3/2020 18:20
1,2,1.0,เอาไรกับคนในพันติ๊บ คำที่พิมเหมือนนางร้ายนางอิ...,22/3/2020 18:18
3,4,1.0,ร้องไห้กับพี่ชานู้ปปี้ #masterchefallstarsthai...,22/3/2020 18:16
7,8,1.0,แงงงงงวีคนี้พิหมีชานนท์คนหิวออก ม่ายยยยยยยยยย ...,22/3/2020 18:14
17,18,1.0,มาตั้งแต่เมื่อไหร่ ยังไม่รู้เลย เพิ่งรู้ตอนเห็...,22/3/2020 18:10


Other class

In [ ]:
master_data.groupby('sentiment').count()

,text_id,text,date
sentiment,,,
0.0,71,71,71
1.0,1744,1744,1744
2.0,1028,1028,1028
3.0,90,90,90
4.0,64,64,64
12.0,1,1,1
22.0,1,1,1


Dropping

In [ ]:
master_drop = master_data[(master_data['sentiment']==0) | (master_data['sentiment']==3) | (master_data['sentiment']==4) | 
                          (master_data['sentiment']==12) | (master_data['sentiment']==22)].index
print("Shape of masterchef before drop {}".format(master_data.shape))
master_data.drop(master_drop,inplace=True)
master_data = master_data.dropna()
print("Shape of masterchef after drop {}".format(master_data.shape))

Shape of masterchef before drop (5000, 4)
Shape of masterchef after drop (2772, 4)


In [ ]:
X_master = master_data['text'].tolist()
y_master = master_data['sentiment'].tolist()

In [ ]:
#text = ' '.join(t for t in X_master)

In [ ]:
#text = text_cleaning(text)

In [ ]:
#text

In [ ]:
# import matplotlib.pyplot as plt
# from pythainlp.tokenize import word_tokenize
# from wordcloud import WordCloud
# from pythainlp.corpus.common import thai_stopwords
# from pythainlp import word_tokenize

# wordcloud = WordCloud(font_path='THSarabunNew.ttf',
#                       stopwords = ' '.join(thai_stopwords()),
#                       background_color="white",
#                       width=1024, # กว้าง
#                       height=768, # ยาว
#                       collocations=False,
#                       regexp=r"[\u0E00-\u0E7Fa-zA-Z']+"
#                       ).generate(' '.join(word_tokenize(text)))
# plt.imshow(wordcloud, cmap=plt.cm.gray, interpolation='bilinear')
# plt.axis("off")
# plt.show()

# ทุ่งเสน่หา Data

In [ ]:
thung_data = pd.read_csv('/content/drive/My Drive/ICT senior project/dataset/ทุ่งเสน่หา17.csv')
thung_data=thung_data.drop(labels='Unnamed: 4', axis=1)
print(thung_data.shape)
thung_data.head()

(5000, 4)


,text_id,sentiment,text,date
0,1,3.0,🌈 เปิด #หารเน็ตฟลิกซ์ 4K 🌈🐳 ดูได้ 1 เดือนกับ 2...,30/3/2020 17:13
1,2,1.0,กูเกลียดฉาก “พี่ไพฑูรย์ รัก จันทร” #ทุ่งเสน่หา...,30/3/2020 15:22
2,3,3.0,#รับสมัครงานพาร์ทไทม์📌📌✔ทำงานผ่านมือถือ✔อายุ18...,30/3/2020 11:13
3,4,3.0,📌 รับสมัครคนตอบแชทลูกค้า- รับกทม.และปริมณฑล- อ...,30/3/2020 10:06
4,5,3.0,เรื่องนี้ต้องขยาย หลังกล้องละครตอนที่ 17#ทุ่งเ...,30/3/2020 09:18


Positive data

In [ ]:
thung_positive = thung_data[thung_data['sentiment'] == 2.0]
print(thung_positive.shape)
thung_positive.head()

(402, 4)


,text_id,sentiment,text,date
8,9,2.0,น่ารักกก #ทุ่งเสน่หาep17 https://t.co/vnNAOK9UrE,29/3/2020 06:25
9,10,2.0,ยิ้มตามเลยย #ทุ่งเสน่หาep17 https://t.co/Eeoit...,29/3/2020 06:24
10,11,2.0,ติดตามน้องกันได้ทางไหนบ้างคะ ใครมีไอจีพ่อแม่น้...,28/3/2020 17:04
12,13,2.0,หวานเว่อ #ทุ่งเสน่หาep17 https://t.co/cXreT9adae,28/3/2020 15:43
14,15,2.0,อีแก้วใจ นี้คือตัวตนมึงใช่ไหม 55555 #ทุ่งเสน่ห...,28/3/2020 15:13


Negative Data

In [ ]:
thung_negative = thung_data[thung_data['sentiment'] == 1.0]
print(thung_negative.shape)
thung_negative.head()

(483, 4)


,text_id,sentiment,text,date
1,2,1.0,กูเกลียดฉาก “พี่ไพฑูรย์ รัก จันทร” #ทุ่งเสน่หา...,30/3/2020 15:22
7,8,1.0,ระหว่างรอฟากฟ้า​คีรี​ดาว🤣🤣//พันตำรวจตรีณวัส​น์...,29/3/2020 08:12
11,12,1.0,สิ่งที่แม่ ป้า และพี่พร่ำสอนเราอยู่เสมอว่า การ...,28/3/2020 16:28
17,18,1.0,พี่มิ่งกวนตีนจริงๆ เหนือสิ่งอื่นใด เกลียดผ้าโพ...,28/3/2020 15:08
18,19,1.0,โอ้ยยยย กุปวดกบาล 🤣🤣 #ทุ่งเสน่หาep17,28/3/2020 15:08


Other class

In [ ]:
thung_data.groupby('sentiment').count()

,text_id,text,date
sentiment,,,
0.0,20,20,20
1.0,483,483,483
2.0,402,402,402
3.0,92,92,92
4.0,4,4,4


Dropping

In [ ]:
thung_drop = thung_data[(thung_data['sentiment']==0) | (thung_data['sentiment']==3) | (thung_data['sentiment']==4) ].index
print("Shape of ทุ่งเสน่หา before drop {}".format(thung_data.shape))
thung_data.drop(thung_drop,inplace=True)
thung_data = thung_data.dropna()
print("Shape of ทุ่งเสน่หา after drop {}".format(thung_data.shape))

Shape of ทุ่งเสน่หา before drop (5000, 4)
Shape of ทุ่งเสน่หา after drop (885, 4)


In [ ]:
X_thung = thung_data['text'].tolist()
y_thung = thung_data['sentiment'].tolist()

# ฉลาดเกมส์โกง Data

In [ ]:
bad_data = pd.read_csv('/content/drive/My Drive/ICT senior project/dataset/ฉลาดเกมส์โกง5.csv')
#bad_data=bad_data.drop(labels='Unnamed: 4', axis=1)
print(bad_data.shape)
bad_data.head()

(5000, 5)


,text_id,sentiment,text,date,tweet_id
0,1,1.0,อย่าคิดที่จะเริ่มทำในเรื่องที่ตัวเองไม่ชอบ เพร...,18/8/2020 21:04,1.295830e+18
1,2,1.0,เนี่ยยยย อันนี้คือสิ่งที่ผู้ใหญ่ปัจจุบันส่วนให...,18/8/2020 20:26,1.295820e+18
2,3,2.0,ซีนนี้ทุกคนปล่อยของจริงๆอะ ไปสุดทุกคนเลยทั้ง พ...,18/8/2020 18:40,1.295790e+18
3,4,1.0,หน่วงไปหมด #ฉลาดเกมส์โกงEP5,18/8/2020 18:36,1.295790e+18
4,5,2.0,ไอซ์ซีนโมโหก็คือไปสุดมาก ตกใจจริง อินมาก จนเชื...,18/8/2020 18:35,1.295790e+18


Positive data

In [ ]:
bad_positive = bad_data[bad_data['sentiment'] == 2.0]
print(bad_positive.shape)
bad_positive.head()

(515, 5)


,text_id,sentiment,text,date,tweet_id
2,3,2.0,ซีนนี้ทุกคนปล่อยของจริงๆอะ ไปสุดทุกคนเลยทั้ง พ...,18/8/2020 18:40,1.295790e+18
4,5,2.0,ไอซ์ซีนโมโหก็คือไปสุดมาก ตกใจจริง อินมาก จนเชื...,18/8/2020 18:35,1.295790e+18
6,7,2.0,ช้อปสินค้าแฟชั่นแบรนด์ชั้นนำได้เลย ที่ https:/...,18/8/2020 18:04,1.295780e+18
7,8,2.0,#ฉลาดเกมส์โกงEP5 กับ #ฉลาดเกมส์โกงEP6 นี่คนละเ...,18/8/2020 18:01,1.295780e+18
8,9,2.0,สนุกนะ5555 #ฉลาดเกมส์โกงEP5,18/8/2020 18:01,1.295780e+18


In [ ]:
bad_negative = bad_data[bad_data['sentiment'] == 1.0]
print(bad_negative.shape)
bad_negative.head()

(434, 5)


,text_id,sentiment,text,date,tweet_id
0,1,1.0,อย่าคิดที่จะเริ่มทำในเรื่องที่ตัวเองไม่ชอบ เพร...,18/8/2020 21:04,1.295830e+18
1,2,1.0,เนี่ยยยย อันนี้คือสิ่งที่ผู้ใหญ่ปัจจุบันส่วนให...,18/8/2020 20:26,1.295820e+18
3,4,1.0,หน่วงไปหมด #ฉลาดเกมส์โกงEP5,18/8/2020 18:36,1.295790e+18
5,6,1.0,ที่สุดแล้ววววว โต้งเอ้ยยยยย 🤬😡😠😤😭😢🥺😩😫😖😣ชอบการแ...,18/8/2020 18:09,1.295780e+18
9,10,1.0,มันก็สะท้อนนะ ถ้าการเมืองดีทุกคนจะเรียนที่ไหนก...,18/8/2020 17:35,1.295780e+18


Other class

In [ ]:
bad_data.groupby('sentiment').count()

,text_id,text,date,tweet_id
sentiment,,,,
0.0,11,11,11,11
1.0,434,434,434,434
2.0,515,515,515,515
3.0,66,66,66,66


Dropping

In [ ]:
bad_drop = bad_data[(bad_data['sentiment']==0) | (bad_data['sentiment']==3) | (bad_data['sentiment']==4) ].index
print("Shape of ฉลาดเกมส์โกง before drop {}".format(bad_data.shape))
bad_data.drop(bad_drop,inplace=True)
bad_data = bad_data.dropna()
print("Shape of ฉลาดเกมส์ after drop {}".format(bad_data.shape))

Shape of ฉลาดเกมส์โกง before drop (5000, 5)
Shape of ฉลาดเกมส์ after drop (949, 5)


In [ ]:
X_bad = bad_data['text'].tolist()
y_bad = bad_data['sentiment'].tolist()

# Combination

In [ ]:
def add_new_list(Xx,yy,new_list):
  if not new_list:
    new_list = []
  for (X,y) in zip(Xx,yy):
    new_entry = [X,y]
    new_list.append(new_entry)
  return new_list

In [ ]:
# train_data = []
# train_data = add_new_list(Xtrain_bad, ytrain_bad, train_data)
# train_data = add_new_list(Xtrain_thung, ytrain_thung, train_data)
# train_data = add_new_list(Xtrain_master, ytrain_master, train_data)
# print(len(train_data))
# train_data[0:5]

In [ ]:
# test_export = pd.DataFrame(test_data,columns=('text','sentiment'))
# test_export.to_csv('test_data.csv')
# train_export = pd.DataFrame(train_data,columns=('text','sentiment'))
# train_export.to_csv('train_data.csv')

In [ ]:
X = X_master + X_thung + X_bad
y = y_master+ y_thung + y_bad

In [ ]:
#X[824]
#text_cleaning('ชานน์ 55555พูดทิ้งท้ายได้กินใจ ทำให้ออกไปอย่างดูดี น่าประทับใจดูเป็น nice guy ถือว่าวางตัวดี สมาร์ทและเป็นที่รักของกรรมการ และผู้แข่งขันคนอื่นๆทำให้บรรยากาศอบอุ่น ซาบซึ้งและเส้นทางอาชีพเชฟสดใสและไม่อยากเปรียบกับนิกเลยบรรยากาศมัน dark มากค่ะ#MasterChefAllStarsThailand https://t.co/sfWpAO94Wv')

In [ ]:
#print(len(X),len(y))

In [ ]:
pd.DataFrame(add_new_list(X,y,[]),columns=('text','sentiment')).to_csv('/content/drive/My Drive/ICT senior project/Dataset_combination/dataset.csv',index_label='id')

# Preprocessing & feature extraction



Text cleaning function

In [ ]:
def text_cleaning(text):
  #emoji list
  
  emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
  #delte hashtag and mention
  text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "",text)
  #delete emoji
  text = re.sub(r"^https://t.co/[A-Za-z0-9]*\s","",text)
  text = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", "", text)
  text = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", "", text)
  text = emoji_pattern.sub(r"", text)

  text = text.replace(" ","")
  text = re.sub(r'(5)\1{2,}(6?){3,}', 'laugh', text)
  
  #text = re.sub(r'\b(?:(?!<laugh>)[^ก-๙])+\b',"",text)
  text = re.sub(r'[!-@[-`{-~]',"",text)
  #text = re.sub("\d+", "", text) #number
  text = normalize(text)

  
  #text = re.sub(r'\W+', '', text) #symbol

  tokens = process_thai(text)


  #deletion of whitespace & one letter text
  i = 0;
  for token in list(tokens):
    if(len(token) == 1 or len(token) == token.count(token[0]) or token == 'xxrep'
    or token == 'xxwrep' or token == ''):
      tokens.pop(i)
      i = i-1
    i = i+1

  return tokens
  return text


Delete stop word

In [ ]:
stop_words = thai_stopwords() 
def delete_stop(word_list):
  clean_word = []
  for w in word_list:
    filtered_sentence = []
    for t in w: 
      if t not in stop_words:
        #t = ''.join(c[0] for c in itertools.groupby(t))
        filtered_sentence.append(t)
    clean_word.append(','.join(filtered_sentence))
  return clean_word

In [ ]:
# X = wordlist
# y = data['sentiment']
# Xtrain_bad, Xtest_bad, ytrain_bad, ytest_bad = train_test_split(X, y, test_size=0.2)

In [ ]:
data = pd.read_csv('dataset.csv',)
data.head()

,id,text,sentiment
0,0,อดเห็นชานนท์ชิมอาหารแล้ว😂 #masterchefallstarst...,1.0
1,1,เอาไรกับคนในพันติ๊บ คำที่พิมเหมือนนางร้ายนางอิ...,1.0
2,2,ต้องคิดถึงซีน’ชิม’ของพี่ชานนท์แน่ๆเลย มีทุกอี...,2.0
3,3,ร้องไห้กับพี่ชานู้ปปี้ #masterchefallstarsthai...,1.0
4,4,ชั้นรักพี่ปอนด์และรักพลังบวกนี้ แล้วทำให้ชั้นน...,2.0


In [ ]:
word_list = [text_cleaning(text) for text in data['text']]

In [ ]:
from pythainlp.tag import pos_tag_sents
pos = pos_tag_sents(word_list,corpus='orchid_ud')
keep_tag = ['VERB', 'ADJ', 'ADV', 'INTJ', 'AUX']
#keep_tag = ['VACT','VATT','ADVN','ADVI','ADVP','ADVS','FIXV','NEG','ADJ','']
pos_list = []
for sent in pos:
  dt_tags = [t[0] for t in sent if (t[1] in keep_tag) or (t[0] == "laugh")  ]
  pos_list.append(dt_tags)
word_list = delete_stop(pos_list)

In [ ]:
print(pos_list[3])
pos[3]

['ร้องไห้']


[('ร้องไห้', 'VERB'),
 ('กับ', 'ADP'),
 ('พี่', 'NOUN'),
 ('ชา', 'NOUN'),
 ('นู้ป', 'NOUN'),
 ('ปี้', 'NOUN')]

In [ ]:
pos_tag_sents([['การถ่ายทำ']],corpus='orchid_ud')

[[('การถ่ายทำ', 'NOUN')]]

In [ ]:
new_list = []
for i in range(len(word_list)):
  new_list.append([data['text'][i], word_list[i]])
b_a = pd.DataFrame(new_list, columns=['before', 'after'])
b_a.head(10)

,before,after
0,อดเห็นชานนท์ชิมอาหารแล้ว😂 #masterchefallstarst...,ชิม
1,เอาไรกับคนในพันติ๊บ คำที่พิมเหมือนนางร้ายนางอิ...,"เหมือน,เข้าไป,กด,เข้าไป,อ่าน,เข้ามา,ไม่เข้าใจ,..."
2,ต้องคิดถึงซีน’ชิม’ของพี่ชานนท์แน่ๆเลย มีทุกอี...,"คิดถึง,laugh"
3,ร้องไห้กับพี่ชานู้ปปี้ #masterchefallstarsthai...,ร้องไห้
4,ชั้นรักพี่ปอนด์และรักพลังบวกนี้ แล้วทำให้ชั้นน...,"บวก,ปอนด์"
5,เชียร์มาตั้งแต่ต้นจนถึงวันนี้ เห็นความตั้งใจ อ...,"แต่ต้น,ความอ่อนน้อม,นนท์,เสมอ"
6,@Pondmcss3 #MasterChefAllStarsThailandปอนด์พัฒ...,
7,แงงงงงวีคนี้พิหมีชานนท์คนหิวออก ม่ายยยยยยยยยย ...,"หิว,ม่าย"
8,เหมือนลุงข้างบ้านขิงกันอะ555555555555555555555...,laugh
9,ลุง : พี่กอล์ฟเอาเบค่อนป่าวให้ยืมแด๊ด : ให้ปะล...,"กอล์ฟ,ให้ยืม,ปะ,ละลุง,แด๊ด,เถียง,laugh"


In [ ]:
# text = ' '.join(t for t in word_list)

In [ ]:
# import matplotlib.pyplot as plt
# from pythainlp.tokenize import word_tokenize
# from wordcloud import WordCloud
# from pythainlp.corpus.common import thai_stopwords
# from pythainlp import word_tokenize

# wordcloud = WordCloud(font_path='THSarabunNew.ttf',
#                       stopwords = ' '.join(thai_stopwords()),
#                       background_color="white",
#                       width=1024, # กว้าง
#                       height=1000, # ยาว
#                       collocations=False,
#                       regexp=r"[\u0E00-\u0E7Fa-zA-Z']+",
#                       max_words = 50
#                       ).generate(' '.join(word_tokenize(text)))
# plt.imshow(wordcloud, cmap=plt.cm.gray, interpolation='bilinear')
# plt.axis("off")
# plt.show()

In [ ]:
# wordcloud.to_file('wordcloud.png')

# Training and Testing Split

In [ ]:
X = word_list
y = data['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
pickle.dump(X_test, open("X_test.pickle", "wb"))
pickle.dump(y_test, open("y_test.pickle", "wb"))

# Feature Extraction (TFIDF)

In [ ]:
tfidf = TfidfVectorizer(analyzer=lambda x:x.split(','))
X_train = tfidf.fit_transform(X)#####
#X = tfidf_fit.transform(word_list)

In [ ]:
tfidf_feature = pd.DataFrame(X_train.todense(), columns=tfidf.get_feature_names()) ####

In [ ]:
print(tfidf_feature.shape)
print(tfidf.get_feature_names())

(4606, 2125)
['', 'cute', 'daddy', 'dark', 'join', 'kingofpressuretest', 'laugh', 'moveon', 'nextlevel', 'setup', 'skip', 'teenagerebellion', 'wetv', 'กด', 'กดดัน', 'กผ้', 'กรง', 'กรร', 'กรรมการตัดสิน', 'กรอ', 'กรอก', 'กรอง', 'กระจาย', 'กระตุก', 'กระวนกระวายใจ', 'กระหม่อม', 'กระโดด', 'กรัก', 'กราบ', 'กราว', 'กริน', 'กริบ', 'กรีดร้อง', 'กรี๊ด', 'กรุ', 'กรุง', 'กลม', 'กลมกล่อม', 'กลับกัน', 'กลับคืน', 'กลับบ้าน', 'กลับมา', 'กลัว', 'กลั้น', 'กลาย', 'กลายเป็น', 'กลิ่น', 'กล้า', 'กวน', 'กวาน', 'กสิ', 'กองทัพ', 'กอด', 'กอล์ฟ', 'กะ', 'กักตัว', 'กัด', 'กัว', 'กั๊บ', 'การคน', 'การคบ', 'การถ่ายทำ', 'การอนุรักษ์', 'การเมือง', 'การเรียนรู้', 'การแสดงออก', 'กำ', 'กำลังใจ', 'กิน', 'กินกัน', 'กินใจ', 'กีดกัน', 'กุ', 'กุม', 'กุ้ง', 'ก้', 'ก้อ', 'ก้อย', 'ก้าว', 'ก๊อนนน', 'ขนลุก', 'ขมวดปม', 'ขยับตัว', 'ขยาย', 'ขยายความ', 'ขยี้', 'ขวัญใจ', 'ของกิน', 'ของจริง', 'ของดี', 'ขอบคุณ', 'ขอโทษ', 'ขอให้', 'ขับ', 'ขาย', 'ขายของ', 'ขายดี', 'ขาว', 'ขำ', 'ขึ้นกับ', 'ขึ้นอยู่กับ', 'ขู่', 'ข้อคิด', 'ข้องใจ', 'ข้าม', 'ข้

In [ ]:
feature_extraction = pd.DataFrame(X_train.todense(),columns=tfidf.get_feature_names())

In [ ]:
feature_extraction.head()

,,"..""",;—;,cute,daddy,dark,join,kingofpressuretest,laugh,moveon,nextlevel,setup,skip,teenagerebellion,wetv,ก.,กด,กดดัน,กผ้,กรง,กรร,กรรมการตัดสิน,กรอ,กรอก,กรอง,กระจาย,กระตุก,กระวนกระวายใจ,กระหม่อม,กระโดด,กรัก,กราบ,กราว,กริน,กริบ,กรีดร้อง,กรี๊ด,กรุ,กรุง,กลม,...,ไม่มีปัญหา,ไม่มีวัน,ไม่มีใคร,ไม่ยอมรับ,ไม่ยาก,ไม่รู้,ไม่รู้จบ,ไม่ว่าง,ไม่สน,ไม่อยู่,ไม่เกี่ยว,ไม่เก่ง,ไม่เข้าใจ,ไม่เคย,ไม่เบา,ไม่เสร็จ,ไม่เห็น,ไม่เอา,ไม่แน่,ไม่แน่ใจ,ไม่ใช่เล่น,ไม่ไหว,ไร้,ไร้ประโยชน์,ไล่,ไว,ไว้เนื้อเชื่อใจ,ไว้ใจ,ไส้กรอก,ไห,ไหม,ไหม้,ไหล,ไหว,ไหว้,ไห้,ไอ,ไอ่,ไอ้,ๆพัฒน์
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.281176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.47512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
feature_extraction[feature_extraction['dark'] != 0]


,,"..""",;—;,cute,daddy,dark,join,kingofpressuretest,laugh,moveon,nextlevel,setup,skip,teenagerebellion,wetv,ก.,กด,กดดัน,กผ้,กรง,กรร,กรรมการตัดสิน,กรอ,กรอก,กรอง,กระจาย,กระตุก,กระวนกระวายใจ,กระหม่อม,กระโดด,กรัก,กราบ,กราว,กริน,กริบ,กรีดร้อง,กรี๊ด,กรุ,กรุง,กลม,...,ไม่มีปัญหา,ไม่มีวัน,ไม่มีใคร,ไม่ยอมรับ,ไม่ยาก,ไม่รู้,ไม่รู้จบ,ไม่ว่าง,ไม่สน,ไม่อยู่,ไม่เกี่ยว,ไม่เก่ง,ไม่เข้าใจ,ไม่เคย,ไม่เบา,ไม่เสร็จ,ไม่เห็น,ไม่เอา,ไม่แน่,ไม่แน่ใจ,ไม่ใช่เล่น,ไม่ไหว,ไร้,ไร้ประโยชน์,ไล่,ไว,ไว้เนื้อเชื่อใจ,ไว้ใจ,ไส้กรอก,ไห,ไหม,ไหม้,ไหล,ไหว,ไหว้,ไห้,ไอ,ไอ่,ไอ้,ๆพัฒน์
824,0.0,0.0,0.0,0.0,0.0,0.384666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#import pickle

pickle.dump(tfidf, open("tfidf.pickle", "wb"))
#pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
#pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

In [ ]:
#feature_extraction.to_csv('feature_extraction.csv')

# Feature selection



In [ ]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
K = round(X_train.shape[0] / 10)
#K = 350
data_selection = SelectKBest(f_classif,k=K)
Xnew = data_selection.fit_transform(X_train, y_train)
Xnew.shape

(3684, 368)

In [ ]:
score = pd.DataFrame([data_selection.scores_, tfidf_feature.columns])
score = score.transpose()
score.sort_values(0, ascending=False).head(10).transpose()

,5,1607,291,297,1591,0,232,980,583,1108
0,105.254,87.5814,68.6046,54.7521,47.6583,47.0587,41.1781,28.1219,26.5119,22.8874
1,laugh,เสมอ,ดี,ดีมาก,เล่น,,ชอบ,ร้องไห้,น่ารัก,สวย


In [ ]:
feature_name_new = list(tfidf_feature.columns[data_selection.get_support(indices=True)])
data_frame = pd.DataFrame(Xnew.todense(), columns=feature_name_new)
data_frame.head()

,,kingofpressuretest,laugh,กรรมการตัดสิน,กรอก,กรัก,กรี๊ด,กลั้น,กล้า,กอด,กอล์ฟ,การอนุรักษ์,การเรียนรู้,กิน,กีดกัน,กุ,ก้อ,ขยาย,ขอบคุณ,ขอให้,ขำ,ขู่,คนรัก,คนรุ่นใหม่,ควรจะ,คาใจ,คิดถึง,คิดได้,ค้าบ,จม,จอง,จอม,จัน,จัล,จำเรียง,จินดา,จี,จีบ,ฉลาด,ชม,...,แสดงออก,แสบ,แห้ง,แอบ,โกหก,โคตร,โดน,โดยเฉพาะ,โบ๊ะ,โอเค,โอ้ย,ใจ,ใจสั่น,ใจหายใจคว่ำ,ใจเสีย,ใช้ได้,ในที่สุด,ใส่,ให้ความรู้,ให้ยืม,ได้คู่,ได้ดิบได้ดี,ไน,ไปดู,ไปหา,ไพฑูรย์,ไฟแรง,ไม่จำเป็น,ไม่ผิด,ไม่สน,ไม่เก่ง,ไม่เข้าใจ,ไม่เคย,ไม่เบา,ไม่เห็น,ไม่แน่ใจ,ไม่ใช่เล่น,ไล่,ไว,ไหล
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.317824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.319533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#support = data_selection.get_support(indices=True)
#support

In [ ]:
data_frame.to_csv('/content/drive/My Drive/ICT senior project/dataset/feature_selection.csv')

In [ ]:
pickle.dump(tfidf, open("tfidf.pickle", "wb"))

In [ ]:
data_frame.shape

(3684, 368)

# Training and Testing data

In [ ]:
dataset = pd.read_csv('feature_selection.csv', index_col=0)
#dataset  = dataset.drop(labels='Unnamed: 1', axis=1)
print(dataset.shape)
dataset.head()

FileNotFoundError: ignored

In [ ]:
X_train = dataset.to_numpy()
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
#clf.predict(X[:2, :])

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train, y_train, cv=10)
scores.mean()

ยังไม่เสร็จ

In [ ]:
clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y, cv=5)
scores

In [ ]:
import pickle
with open('clf.pkl', 'wb') as file:
  pickle.dump(clf, file)

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
from sklearn import metrics
confusion = metrics.confusion_matrix(y_test, predictions)

In [ ]:
from sklearn import svm
svm_model = svm.SVC()
svm_model.fit(X_train,y_train)

In [ ]:
svm_pred = svm_model.predict(X_test)
svm_confusion = metrics.confusion_matrix(y_test, svm_pred)
print(svm_model.score(X_test,y_test))
print(svm_confusion)

In [ ]:
X.shape

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,svm_pred))

In [ ]:
new = tfidf.transform(['แย่'])
print(new.shape)
new = new.todense()
print(new)

sum = 0
#new = new[0, data_selection.get_support(indices=True)]

#new.tolist()
print(new)
#new[2196]
#new[data_selection.get_support(indices=True)]
#new = new.todense()
